<a href="https://colab.research.google.com/github/Vimp17/py/blob/main/bart_finetuning_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set(palette='summer')

In [2]:
!pip install accelerate -U

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
!pip install transformers

In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00


In [6]:
import transformers
from datasets import load_dataset
import evaluate

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

In [8]:
billsum = load_dataset("billsum", split="ca_test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/91.8M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/15.8M [00:00<?, ?B/s]

ca_test-00000-of-00001.parquet:   0%|          | 0.00/6.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [9]:
billsum

Dataset({
    features: ['text', 'summary', 'title'],
    num_rows: 1237
})

In [10]:
billsum = billsum.train_test_split(test_size=0.1)

In [11]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1113
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 124
    })
})

In [12]:
tokenizer = transformers.AutoTokenizer.from_pretrained("ainize/bart-base-cnn")

tokenizer_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["text"], max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/1113 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [15]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained("ainize/bart-base-cnn")

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [16]:
data_collator = transformers.DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [17]:
training_args = transformers.Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=2,
    )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = transformers.Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-18-199256a528cd>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Seq2SeqTrainer(


In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,1.963167
2,2.175500,1.912211


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=558, training_loss=2.158746042559224, metrics={'train_runtime': 655.0178, 'train_samples_per_second': 3.398, 'train_steps_per_second': 0.852, 'total_flos': 1357273356042240.0, 'train_loss': 2.158746042559224, 'epoch': 2.0})

###  Предсказания на тестовом множестве

In [20]:
text_example = billsum["test"]['text'][0]
print(text_example)

The people of the State of California do enact as follows:


SECTION 1.
(a) The Legislature finds and declares the following:
(1)
A new form of work has proliferated in which individuals work by the job through an electronic platform, such as the Internet or telephone. These individuals are hired through these hosting platforms to perform short-term work, usually of a day or less, for multiple customers.
(2)
These individuals are not treated by the hosting platforms as employees and do not receive the benefit of state labor protection laws such as minimum wage, unemployment insurance, and workers’ compensation. The platforms treat these individuals as independent contractors and even though they perform work for multiple clients, usually individual people, the individuals securing work through a hosting platform are normally prohibited from negotiating the terms of their services. Instead, the hosting platforms dictate the terms and take a considerable portion of the amount paid for se

In [21]:
input_ids = tokenizer.encode(
    text_example,
    return_tensors="pt",
    max_length=1024,
    truncation=True,
    ).to(device)

In [22]:
input_ids.shape

torch.Size([1, 1024])

In [23]:
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    max_length=142,
    min_length=56,
    num_beams=4,
)

In [24]:
summary_text_ids

tensor([[    2,     0,  9089, 15528,   488,  2730,  7396,    41,  2222,  9254,
            54, 14023,    44,    48, 22930,  2088,   518,     6,    17,    46,
            25,  6533,     6,     7,  4949,    11,   333,  1940,    19,  2098,
             7,    65,    50,    55,  5162,  4818,     6,   217,     6,    53,
            45,  1804,     7,     6,    10, 22903,   544,     6,    41,  3742,
          6494,  1082,     6,    50,    97,  3742,    12,   805,  1082,     6,
            25,  6533,     4, 50118,   713,  1087,    74, 29080,     5,   331,
           641,     9,  6338,     8,  7055,     7, 14244,     5,  1713,     9,
          2222,  9617,     4,    20,  1087,    74,    67, 29080,     5,   194,
           641,     9,  7653,  4702,     7, 14244,     8, 14244,     5,   304,
             9,  3742,     8,  7377,   518,    30,  2222,  9617,     6,    25,
         17966,     4,     2]], device='cuda:0')

In [25]:
decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)

In [26]:
len(decoded_text), len(text_example)

(594, 10015)

In [27]:
decoded_text

'Existing law authorizes an independent contractor who performs “supportive services,” as defined, to engage in group activity with respect to one or more hosting platforms, including, but not limited to, a dispatch service, an Internet Web site, or other Internet-based site, as defined.\nThis bill would authorize the State Department of Labor and Industries to regulate the activities of independent contractors. The bill would also authorize the state Department of Consumer Affairs to regulate and regulate the use of Internet and telephone services by independent contractors, as specified.'

In [28]:
summaries = []

for text in tqdm(billsum["test"]['text']):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        max_length=1024,
        truncation=True,
        ).to(device)

    summary_text_ids = model.generate(
        input_ids=input_ids,
        bos_token_id=model.config.bos_token_id,
        eos_token_id=model.config.eos_token_id,
        max_length=142,
        min_length=56,
        num_beams=4,
    )

    decoded_text = tokenizer.decode(summary_text_ids[0], skip_special_tokens=True)
    summaries.append(decoded_text)

  0%|          | 0/124 [00:00<?, ?it/s]

## Считаем качество

### ROUGE

In [29]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e0ef6c079ae7b443c5e1980a02c936df0f7450ff86a2d706f6d4c8aad9805b9b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [30]:
rouge = evaluate.load('rouge')

In [31]:
%%time
results = rouge.compute(
        predictions=summaries,
        references=billsum["test"]['summary']
    )

CPU times: user 6.4 s, sys: 24.5 ms, total: 6.42 s
Wall time: 7.64 s


In [32]:
results

{'rouge1': 0.3399550881984635,
 'rouge2': 0.18715523681655766,
 'rougeL': 0.22940936004838447,
 'rougeLsum': 0.2920608235651465}